In [1]:
import boto3
import time
from SeekerTruther import Seeker, Truther
import KnowledgeGraph as kg
from SeekerTruther import Seeker, Truther
from RewardFunction import RewardFunction
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
textract = boto3.client(
    'textract',
    aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
    aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
    region_name='eu-west-2'
)

# Specify S3 bucket and document details
bucket_name = 'knowledge-graph-test-examples'
document_name = 'KPMG-TL-FDD-Report-Aug-2020.pdf'

In [3]:
# Step 1: Start the Document Analysis Job
response = textract.start_document_analysis(
    DocumentLocation={'S3Object': {'Bucket': bucket_name, 'Name': document_name}},
    FeatureTypes=['TABLES', 'LAYOUT']
)

job_id = response['JobId']
print(f"Started Textract job with JobId: {job_id}")

# Step 2: Poll for Job Completion
while True:
    status = textract.get_document_analysis(JobId=job_id)
    status_value = status['JobStatus']
    if status_value in ['SUCCEEDED', 'FAILED']:
        print(f"Job status: {status_value}")
        break
    print("Waiting for job to complete...")
    time.sleep(5)

all_blocks = []
if status_value == 'SUCCEEDED':
    # Step 3: Retrieve and Process Results
    next_token = None
    while True:
        response = textract.get_document_analysis(JobId=job_id, NextToken=next_token) if next_token else textract.get_document_analysis(JobId=job_id)
        all_blocks.extend(response['Blocks'])
        if 'NextToken' not in response:
            break
        next_token = response['NextToken']

Started Textract job with JobId: 31466b1a2898aebd4616bf6ce577fa4b1d80bab72ba3dd638fede03cb54e8bf2
Waiting for job to complete...
Waiting for job to complete...
Waiting for job to complete...
Waiting for job to complete...
Waiting for job to complete...
Waiting for job to complete...
Waiting for job to complete...
Waiting for job to complete...
Waiting for job to complete...
Waiting for job to complete...
Waiting for job to complete...
Job status: SUCCEEDED


In [4]:
all_block_types = set()
for b in all_blocks:
    all_block_types.add(b['BlockType'])
    

In [ ]:
for b in all_blocks:
    if b['BlockType'] == 'TABLE':
        if len(b['Relationships']) > 3:
            print(b['Relationships'])
            break

[{'Type': 'CHILD', 'Ids': ['65bbf452-08e4-4df0-b85c-03b75db1dbe2', '91eef293-a279-4ccb-9b25-704fc8e48e0b', 'bd9ba013-2446-4d19-923b-3016e11a1fbe', 'a50fc03f-7d07-4956-8133-c9c095160d23', 'f4a30e91-6831-4edb-9bae-607925680253', '9371b0c3-205d-4418-b759-8795fe030455', '6c209ebf-0f4b-4e80-94f9-a790559c5abf', '110d7644-f89a-4efd-9449-e2b502f1124b', '457e70d2-7c84-418a-86a6-655ca353ee0e', '655947d0-c7b3-450a-b812-2a3326906be1', '733f23b0-645c-488f-a73b-f3c18348533b', 'c2c1fb50-a8fa-45e7-bb23-61d1a9465944']}, {'Type': 'MERGED_CELL', 'Ids': ['7c13b269-3c93-4eac-86fd-aea1b72bf769', '09c9059b-74f7-4d34-baa9-8a8b7e9a81a6']}, {'Type': 'TABLE_TITLE', 'Ids': ['97352a7e-6031-4e2a-a223-b07d34b89bf8']}, {'Type': 'TABLE_FOOTER', 'Ids': ['17fb9e8b-08c0-4020-ac22-9f753f9ce46e']}]


In [6]:

kg = kg.KnowledgeGraph(is_seeker = lambda x: x.isnumeric(), is_truth = lambda x: x.isnumeric(), textract_obj = all_blocks)

def seeker_and_truther_have_identical_value(seeker, truther, knowledge_graph):
    return knowledge_graph.nodes[seeker]['Text'] == knowledge_graph.nodes[truther]['Text']

calculated_probabilities = kg.calculate_probabilities(attribute_list = [seeker_and_truther_have_identical_value], labels = [])
reward_function = RewardFunction(calculated_probabilities)






KeyError: 'Relationships'